In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import scipy as sp
from SimISR.utilFunctions import readconfigfile,makeconfigfile,MakePulseDataRepLPC
from SimISR.IonoContainer import IonoContainer,MakeTestIonoclass
from ISRSpectrum.ISRSpectrum import ISRSpectrum
from SimISR.analysisplots import analysisdump
import seaborn as sns

/Users/Bodangles/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [ ]:
rep1=1e4
Npnts=2**7
xin = sp.random.randn(rep1,Npnt)+1j*sp.random.randn(rep1,Npnt)

In [ ]:
spfreq=50e3
nspec=128
ISpec_ion = ISRSpectrum(centerFrequency = 449e6, nspec=nspec, sampfreq=spfreq,dFlag=True)

species=['O+','e-']
databloc = np.array([[1.66e10,1e3],[1.66e10,2.5e3]])

f,cur_spec = ISpec.getspecsep(databloc,species)


cur_pulse_data = MakePulseDataRepLPC(pulse,cur_spec,20,rep1,numtype = sp.complex128)